# **Link do driva ze obrazkami:**
[drive](https://drive.google.com/drive/folders/1lz0DJowtewvzcATP32UM0JThLu-wB2Q-?usp=drive_link)


---

Najlepszy  stosunek czasu generowania do jakości:

[14s](https://drive.google.com/drive/folders/1OvsbGroJJ3tky7DC0dLPxfasYtkDYgI3?usp=drive_link) ~ "misri/cyberrealisticXL_v11VAE" + guidance_scale=6.0 + inference_steps=40 + DeepCache [cache_interval=3.0 + cache_branch=0.0]

Ewentualnie:

[35s](https://drive.google.com/drive/folders/14ViCTkZA4VshPot1yl8hXhIVPXCB69gE?usp=sharing) ~ "misri/cyberrealisticXL_v11VAE" + guidance_scale=6.0 + inference_steps=40

[10,7s](https://drive.google.com/drive/folders/1A9KGM9yzB5OklU619hDdWy56SJneov_N?usp=drive_link) ~ "misri/cyberrealisticXL_v11VAE" + guidance_scale=5.0 + inference_steps=35 + DeepCache [cache_interval=4.0 + cache_branch=0.0]

[1s](https://drive.google.com/drive/folders/1_bCy1AUXsFg_54idDnSHUokXYO_7ALjR?usp=drive_link) ~ "stabilityai/sdxl-turbo" + guidance_scale=0.0 + inference_steps=3



In [ ]:
!pip install -q diffusers transformers accelerate
from diffusers import AutoPipelineForText2Image
import torch
import time
import random
import matplotlib.pyplot as plt

**Modele:**

---
**Najlepszy:**

`"misri/cyberrealisticXL_v11VAE"`

[CivitAi](https://civitai.com/models/15003/cyberrealistic)

[Huggin Face](https://huggingface.co/misri/cyberrealisticXL_v11VAE)

---

***Eksperymentowałem jeszcze z:***

`"philz1337x/cyberrealistic-v4.2"` - ten sam model tylko oparty na SD 1.5. Szybszy, ale znacznie gorsza jakość

`"stabilityai/sdxl-turbo"` - zwykły turbo, zajebiście szybki ale nie potrafiłem wyciągnąć go na akceptowalną jakość



In [ ]:
model = "misri/cyberrealisticXL_v11VAE"
pipeline_text2image = AutoPipelineForText2Image.from_pretrained(model, torch_dtype=torch.float16, variant="fp16")
pipeline_text2image = pipeline_text2image.to("cuda")
# use_safetensors=True

**Podpięcie Driva do zapisu obrazków**

In [3]:
import os
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


**Parametry generowanych obrazków**

In [4]:
Object = "Dog" # Generowny obiekt

breeds = ["Golden Retriever",
          "Bulldog"]

positions = ["Sitting",
             "Walking",
             "Running",
             "Sleeping",
             "Standing"]

places = ["City park",
          "Beach",
          "River",
          "Forrest",
          "Countryside",
          "Mountains",
          "Living room",
          "Bedroom",
          "Kitchen",
          "Farmland",
          "Snowy",
          "Fall",
          "Rainy",
          "Windy"]

# Prompty
rest_of_prompt = "background, 8k, high quality, ultradetailed, ultrarealistic, colorful details, subsurface scattering, natural lighting, amazing composition, amazing textures,soft light, realistic background, realistic eyes, detailed eyes, detailed paws, detailed legs"
neg = "worst quality, low quality, painting, anime, unrealistic, normal quality, lowres, low details, wrong numer of paw's phalanges, bad paws, bad iris, bad eyes, grayscale, humans, starfish, clothes, undetailed paws, scales, undetailed fur, unrealistic fur, wrong number of paws, undetailed tail, undetailed legs, other creatures, highly blured background, looking straight to camera, undetailed paws, too many paws, collar, harness, toys, ball, undetailed eyes, unrealistic eyes, bad anatomy, bad body, bad face, bad teeth, bad arms, wrong number of tails, blured eyes, bad legs, fantasy elements, wrong proportions, deformities:2.5"

# Najlepsze ustawienia:
guidance_scale = 6.0
inference_steps = 40
cache_interval = 3.0
cache_branch = 0.0

**Funkcja do pokazywania 9 wygenerowanych obrazków**

(przydatne w przypadku eksperymentowania z usatwieniami)

In [6]:
def show9(images):
  rows = 3
  cols = 3
  plt.figure(figsize=(10, 10))
  for i, img in enumerate(images):
      plt.subplot(rows, cols, i+1)
      plt.imshow(img)
      plt.axis('off')
      if i >= (rows * cols) - 1:
        break
  plt.tight_layout()
  plt.show()

**Sposób na przyspieszenie generowania obrazków (DeepCache)**

[Huggin Face - coś więcej o DeepCache](https://huggingface.co/docs/diffusers/main/en/optimization/deepcache)

`cache_interval` - im wyższy tym niższa jakość, szybszy czas generowania

`cache_branch_id` - im niższy tym niższa jakość, szybszy czas generowania

Dla mnie najlepiej sprawdzały się parametry odpowiednio 3 i 0.

In [ ]:
!pip install DeepCache
from DeepCache import DeepCacheSDHelper
helper = DeepCacheSDHelper(pipe=pipeline_text2image)
helper.set_params(
    cache_interval=cache_interval,
    cache_branch_id=cache_branch,
)
helper.enable()

**Generowanie obrazków**

Zdjęcia generuje w ten sposób, że w prompcie zmienia się jedynie rasa `(breeds[x])`, pozycja w której znajduje się pies `(positions[y])` i miejsce w którym się znajduje `(places[random_numbers[y]])`.

Dla każdej rasy psów jest generowane 5 zdjęć .
Pierwsza pętla przechodzi po wszystkich rasach psów, druga po wszystkich pozycjach. Miejsca w których znajduje się pies, dla każdego promptu są losowo wybierane z listy. Wszystko po to aby data set miał jak największą róźnorodność obrazków.

**Do eksperymentowania:**

(na końcu wyświetla 9 z wygenerowanych obrazków i printuje informacje o czasie generowania)

In [ ]:
images = []
start_time = time.time()
for x, breed in enumerate(breeds):
  numbers = list(range(len(positions)))
  random.shuffle(numbers)
  random_numbers = numbers[:5]
  for y, position in enumerate(positions):
    prompt = f"{Object}, {breeds[x]}, {positions[y]}, {places[random_numbers[y]]}, {rest_of_prompt}"
    image = pipeline_text2image(prompt=prompt, guidance_scale=guidance_scale, num_inference_steps=inference_steps, negative_prompt=neg).images[0]
    images.append(image)
end_time = time.time()
print(f"Czas wygenerowania {(breeds*positions)} obrazków: {end_time - start_time}\nCzas wygenerowania jednego obrazka: {(end_time - start_time)/(breeds*positions)}")

show9(images)

**Do masowego generowania:**

(zapisuje zdjęcia oraz dane generowanych obrazków na drive, [przykład](https://drive.google.com/drive/folders/1da7myV42vxOh-Fx0V4oXxD1KvhykuoCN?usp=drive_link))

In [ ]:
file_path = "/gdrive/MyDrive/Projekt/Próba/" # Miejsce zapisu plików
file = open(f"{file_path}Data - {Object}.txt", 'w')
file.writelines('torch_dtype=torch.float16, variant="fp16", generated on "cuda"\n') #
file.writelines(f"Model = {model}\n\nprompt = {rest_of_prompt}\n\nnegative_prompt = {neg}\n\nguidance_scale = {guidance_scale}\nnum_inference_steps = {inference_steps}\n\n")
# Jeśli używa się DeepCache odkomentować ↓
file.writelines(f"DeepCache:\ncache_interval = {cache_interval}\ncache_branch_id = {cache_branch}\n\nObject: {Object}\n") #
i=1
start_time = time.time()

for x, breed in enumerate(breeds):
  numbers = list(range(len(positions)))
  random.shuffle(numbers)
  random_numbers = numbers[:5]
  for y, position in enumerate(positions):
    prompt = f"{Object}, {breeds[x]}, {positions[y]}, {places[random_numbers[y]]}, {rest_of_prompt}"
    image = pipeline_text2image(prompt=prompt, guidance_scale=guidance_scale, num_inference_steps=inference_steps, negative_prompt=neg).images[0]
    image.save(f"{file_path}{Object} {i}.jpg")
    file.writelines(f"{i} - {breeds[x]}, {positions[y]}, {places[random_numbers[y]]}\n")
    i=i+1

end_time = time.time()
file.writelines(f"\nCzas wygenerowania {(len(breeds)*len(positions))} obrazków: {end_time - start_time}\nCzas wygenerowania jednego obrazka: {(end_time - start_time)/(len(breeds)*len(positions))}")
file.close()